In [3]:
import pandas as pd

df = pd.read_csv('./data/evaluation_data.csv')
df

,question,answer,type
0,What is the deadline for transfer students to ...,The deadline for transfer students to apply fo...,test
1,Who is authorized to request verification of i...,University Police Officers have the authority ...,train
2,What does a probationary appointment at SUNY B...,A probationary appointment at SUNY Brockport i...,train
3,What characteristics do you seek in a Peer Men...,We're excited to find Peer Mentors who are pro...,train
4,What are some effective strategies for prepari...,To prepare effectively for an interview at SUN...,test
...,...,...,...
2340,Are there any internships available in the fie...,Absolutely! SUNY Brockport provides a wealth o...,test
2341,How can I view the Group Exercise Schedule?,You can view our Group Exercise Schedule by vi...,test
2342,Where can I get help with my resume and cover ...,You can get professional help with your resume...,test
2343,"How do I enroll in the Equity, Diversity, and ...",We're thrilled to hear about your interest! Yo...,train


In [4]:
df['type'].value_counts()

type
test     1345
train    1000
Name: count, dtype: int64